In [1]:
!pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
import pandas as pd
df = pd.read_csv('/content/spam.csv')
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [5]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [6]:
747/4825

0.15481865284974095

In [7]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [8]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [9]:
df_ham_downsmapled = df_ham.sample(df_spam.shape[0])
df_ham_downsmapled.shape

(747, 2)

In [10]:
df_balanced = pd.concat([df_spam, df_ham_downsmapled])
df_balanced.shape

(1494, 2)

In [11]:
df_balanced['Category'].value_counts()

spam    747
ham     747
Name: Category, dtype: int64

In [12]:
df_balanced.sample(5)

,Category,Message
4373,spam,Ur balance is now £600. Next question: Complet...
2708,spam,Great NEW Offer - DOUBLE Mins & DOUBLE Txt on ...
3446,ham,wiskey Brandy Rum Gin Beer Vodka Scotch Shampa...
3062,ham,What u talking bout early morning? It's almost...
2691,spam,sports fans - get the latest sports news str* ...


In [13]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(10)

,Category,Message,spam
3874,ham,"Happy or sad , one thing about past is- ""Its n...",0
4838,ham,....photoshop makes my computer shut down.,0
2480,spam,Sppok up ur mob with a Halloween collection of...,1
823,spam,"SMSSERVICES. for yourinclusive text credits, p...",1
3991,spam,(Bank of Granite issues Strong-Buy) EXPLOSIVE ...,1
4267,ham,The greatest test of courage on earth is to be...,0
1019,ham,Nope watching tv at home... Not going out. V b...,0
4355,spam,important information 4 orange user 0789xxxxxx...,1
2800,ham,I've told him that i've returned it. That shou...,0
3885,spam,A link to your picture has been sent. You can ...,1


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'], stratify=df_balanced['spam'])

In [15]:
X_train.head(5)

527     Today's Offer! Claim ur £150 worth of discount...
3456    No need lar. Jus testing e phone card. Dunno n...
3176    UR GOING 2 BAHAMAS! CallFREEFONE 08081560665 a...
5070    Hai dear friends... This is my new &amp; prese...
1803                         Ok lor thanx... Ü in school?
Name: Message, dtype: object

In [18]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [20]:
def get_sentence_embeding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleyball game tomorrow?"
    ])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435172 , -0.51327294, -0.88845766, ..., -0.7474892 ,
        -0.7531475 ,  0.91964495],
       [-0.8205969 , -0.5209497 , -0.95245343, ..., -0.8804193 ,
        -0.7053686 ,  0.8252003 ]], dtype=float32)>